In [4]:
# importing all the data 
import pandas as pd
import numpy as np
import os,datetime
from sklearn.linear_model import LinearRegression
from nepali_date_converter import english_to_nepali_converter

intial_data = pd.read_excel('merged_data/ADBL.xlsx',skiprows=0)
data = pd.DataFrame(columns=intial_data.columns)
market_data = pd.read_excel('merged_data/NEPSE.xlsx',skiprows=0)
# # # appending all the company data in single excel sheet
for file in os.listdir('merged_data'):
    company_data = pd.read_excel(f'merged_data/{file}',skiprows=0)
    #mergin the stock price with nepse index
    merged_data  = market_data.merge(company_data,on='Date',how='outer')[['Symbol_y','Date','Close_y','Close_x','Percent Change_x','Percent Change_y']].dropna(how='any').sort_values('Date')
    merged_data.rename(columns={'Close_y' : 'company' ,'Close_x' : 'nepse','Symbol_y':'Script','Percent Change_x':'Rm','Percent Change_y':'Rj'},inplace=True)
    merged_data['rm'] = merged_data.nepse.pct_change()*100
    merged_data['rj'] = merged_data.company.pct_change()*100
    merged_data['no_of_trading_days'] = range(len(merged_data))
    # getting the event date from the csv
    event_data = pd.read_excel('event_dates.xlsx')[['Date','Quote','EPS','Remarks']].dropna(how='any',axis = 0,subset=['Date','Quote','EPS'])
    data = pd.concat([merged_data,data])
data= data[['Script','Date','nepse','company','rm','rj','no_of_trading_days']] #this is the common database for all company
ar_data = pd.DataFrame(columns=['Company',"Event_date",'trading_date',10,9,8,7,6,5,4,3,2,0,0,-0,-2,-3,-4,-5,-6,-7,-8,-9,-10])



In [3]:
company_data

,RBCL,2015-06-23 00:00:00,5098,5098.1,5098.2,5098.3,-2,214
0,RBCL,2015-06-24,5098.0,5098.0,5098.0,5098.0,0,25
1,RBCL,2015-07-01,5199.0,5199.0,5096.0,5096.0,-0.04,285
2,RBCL,2015-07-02,4995.0,4995.0,4995.0,4995.0,-1.98,51
3,RBCL,2015-07-06,5000.0,5000.0,5000.0,5000.0,0.1,103
4,RBCL,2015-07-12,5100.0,5100.0,5100.0,5100.0,2,41
...,...,...,...,...,...,...,...,...
1730,RBCL,2024-04-15,13299.0,13299.0,13061.0,13195.0,1.09 %,130
1731,RBCL,2024-04-16,13100.0,13194.0,12990.0,12990.0,-1.55 %,300
1732,RBCL,2024-04-18,13000.0,13050.0,12933.0,12933.0,-0.44 %,396
1733,RBCL,2024-04-21,13000.0,13000.0,12901.0,13000.0,0.52 %,183


In [7]:
data = pd.read_excel('data_shortcut.xlsx')
event_data = pd.read_excel('event_data.xlsx')
# eps_data = pd.read_excel('eps.xlsx')
ar_data  = pd.read_excel('ar_data.xlsx')
# final_data = pd.read_excel('final.xlsx')

In [6]:
# modification of event_data  dataframe
event_data['trading_day'] = None

# locating the next trading day after news and appending it to the datframe

for row,values in event_data.iterrows():
    day =values['Date']
    company_name = values['Quote']
    while len(data[(data['Date'] == day) & (data['Script'] == company_name)]) != 1:
        print(day,company_name)
        day += datetime.timedelta(days= 1)
    event_data.at[row,'trading_day'] = day
    print(day,company_name,'found')

event_data.to_excel('event_shortcut.xlsx',index=False)


2023-08-05 00:00:00 ADBL
2023-08-06 00:00:00 ADBL found
2023-04-21 00:00:00 ADBL
2023-04-22 00:00:00 ADBL
2023-04-23 00:00:00 ADBL found
2023-01-21 00:00:00 ADBL
2023-01-22 00:00:00 ADBL
2023-01-23 00:00:00 ADBL found
2022-10-24 00:00:00 ADBL
2022-10-25 00:00:00 ADBL
2022-10-26 00:00:00 ADBL
2022-10-27 00:00:00 ADBL
2022-10-28 00:00:00 ADBL
2022-10-29 00:00:00 ADBL
2022-10-30 00:00:00 ADBL
2022-10-31 00:00:00 ADBL found
2022-08-04 00:00:00 ADBL found
2022-04-20 00:00:00 ADBL found
2022-01-21 00:00:00 ADBL
2022-01-22 00:00:00 ADBL
2022-01-23 00:00:00 ADBL found
2021-10-31 00:00:00 ADBL found
2021-08-11 00:00:00 ADBL found
2021-04-27 00:00:00 ADBL found
2021-01-28 00:00:00 ADBL found
2020-11-13 00:00:00 ADBL
2020-11-14 00:00:00 ADBL
2020-11-15 00:00:00 ADBL
2020-11-16 00:00:00 ADBL
2020-11-17 00:00:00 ADBL
2020-11-18 00:00:00 ADBL found
2020-08-16 00:00:00 ADBL found
2020-05-12 00:00:00 ADBL found
2020-02-04 00:00:00 ADBL found
2019-11-13 00:00:00 ADBL found
2019-08-06 00:00:00 ADBL foun

In [75]:
# event_data['trading_day'] = pd.to_datetime(event_data['trading_day'])
# data_with_event_date = data.merge(event_data,right_on = ['Quote','trading_day'],left_on=['Script','Date'],how='left').drop(['trading_day','Quote'],axis=1)
# data_with_event_date.rename(columns={'Date_x':'Date','Date_y':'Event_date'},inplace=True)
# data_with_event_date.to_excel('data_with_event_date.xlsx', index = False)


In [76]:
# #ar_data is already loaded above


# data_event_only = data_with_event_date[(data_with_event_date['Event_date'].notnull()) & (data_with_event_date['no_of_trading_days']>=250)]
# data_with_event_date['er'] = np.nan
# for ind,rows in data_event_only.iterrows():
#     regression_period = data_with_event_date[ind-250:ind-10].dropna(subset=['rm','rj'],axis=0)
#     event_window_period= data_with_event_date[ind-10:ind+11]
#     if event_window_period.Script.nunique()==1:
#         model = LinearRegression()
#         x = np.array(regression_period['rm']).reshape(-1,1)
#         y = np.array(regression_period['rj']).reshape(-1,1)
#         model.fit(x,y)
#         to_predict = np.array(event_window_period['rm']).reshape(-1,1)
#         # event_window_period['er'] = model.predict(to_predict)
#         data_with_event_date.loc[ind-10:ind+10,'er'] = model.predict(to_predict)
#         predictions = model.predict(to_predict).flatten()
#         other_detail = np.array([rows['Script'],rows['Event_date'],rows['Date']])
#         final_detail = np.concatenate([other_detail,predictions],axis =0)
#         ar_data.loc[len(ar_data)] =  final_detail
#     else:
#         print(ind)
# data_with_event_date['Ar'] = data_with_event_date['rj']-data_with_event_date['er']
# ar_data.dropna(subset=['Company'],axis = 0,inplace=True)


1894
9337
14645
16801
18733
21006
23243
25505
29204
31482
34363
36037
38314
40415


In [77]:
# # getting the final dataset for analysis
# ar_data['quarter'] = ar_data.Event_date.dt.quarter
# quarter_mapping = {4:'q1',1:'q2',2:'q3',3:'q4'}
# ar_data['quarter'] = ar_data['quarter'].map(quarter_mapping)
# ar_data['fiscal_year'] = None
# for index,rows  in ar_data.iterrows():
#     if rows['quarter'] not in  ['q1']:
#         ar_data.at[index,'fiscal_year'] = f'0{rows.Event_date.year + 56-2000}/0{rows.Event_date.year + 57-2000}'
#     else:
#         ar_data.at[index,'fiscal_year'] = f'0{rows.Event_date.year + 57-2000}/0{rows.Event_date.year + 58-2000}'
# nepali_dates = []
# for date in ar_data.Event_date:
#     splitted = str(date).split('-')
#     nepali = english_to_nepali_converter(int(splitted[0]),int(splitted[1]),int(splitted[2][:2]))
#     nepali_dates.append(nepali)
# ar_data['Nepali_date'] = nepali_dates
# final_data = ar_data.merge(event_data, left_on = ['Company','Event_date'],right_on = ['Quote','Date'], how = 'left')
# final_data.drop(columns=['Date','Quote','trading_day'],inplace=True)
# bull1= (final_data['Event_date'] >= '2011-07-01') & (final_data['Event_date'] <= '2016-07-03')
# bull2= (final_data['Event_date'] >= '2019-02-03') & (final_data['Event_date'] <= '2021-08-1')
# bear1= (final_data['Event_date'] >= '2016-07-03') & (final_data['Event_date'] <= '2019-02-03')
# bear2= final_data['Event_date'] >= '2021-08-1'

# final_data.loc[bull1, 'market_condition'] = 'bm1'
# final_data.loc[bull2, 'market_condition'] = 'bm2'
# final_data.loc[bear1, 'market_condition'] = 'dm1'
# final_data.loc[bear2, 'market_condition'] = 'dm2'


In [17]:
import pandas as pd
final_data = pd.read_excel('final.xlsx')
final_data.drop(columns=['Date','Quote','trading_day'],inplace=True)


In [16]:
final_data.to_excel('final.xlsx',index=False)

In [18]:
final_data

,Company,Event_date,trading_date,10,9,8,7,6,5,4,...,-7,-8,-9,-10,quarter,fiscal_year,Nepali_date,EPS,Remarks,market_condition
0,GBIME,2015-05-05,2015-05-25,0.171653,0.170793,-0.120268,-0.120812,0.203827,-0.120539,-0.023736,...,0.791472,0.075088,0.141175,0.470334,q3,071/072,2072-01-22,21.54,bad,bm1
1,GBIME,2015-07-28,2015-07-28,-0.128690,0.115249,-0.250401,0.114991,0.479223,1.080743,0.110680,...,0.538721,2.414956,1.818918,-0.447886,q4,071/072,2072-04-12,20.07,bad,bm1
2,GBIME,2015-11-09,2015-11-09,-0.842146,-3.291480,-0.972536,0.677734,-0.576322,0.220259,-0.819289,...,0.325611,-1.178781,-1.317326,-1.570491,q1,072/073,2072-07-23,20.35,good,bm1
3,GBIME,2016-01-27,2016-01-27,0.533939,0.711237,1.974199,0.010665,-0.275909,-0.230683,-0.339898,...,0.870813,-0.128976,-0.219745,-0.629813,q2,072/073,2072-10-13,24.75,good,bm1
4,GBIME,2016-04-25,2016-04-25,0.317368,-0.013774,0.294934,1.429909,0.185507,1.589505,0.051618,...,-0.578512,0.687856,0.738755,-0.047666,q3,072/073,2073-01-13,22.33,bad,bm1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,RBCL,2022-02-10,2022-02-10,-1.626068,0.060505,4.238217,-0.645930,0.725633,-0.647936,-1.678141,...,0.996286,0.652319,-0.719467,-2.539541,q2,078/079,2078-10-27,145.75,bad,dm2
624,RBCL,2022-05-12,2022-05-12,-0.644202,0.263024,-0.871216,-2.183410,-1.443305,-1.713027,0.887144,...,0.723170,0.414264,-0.726447,-2.392598,q3,078/079,2079-01-29,171.36,good,dm2
625,RBCL,2022-08-16,2022-08-16,-0.308047,-1.796326,-1.650924,0.507801,-0.845908,-2.094230,-0.291102,...,-0.904224,-0.843967,-1.677963,0.193463,q4,078/079,2079-04-31,177.58,good,dm2
626,RBCL,2022-11-15,2022-11-15,3.179424,0.428981,-0.845146,2.394906,-0.312847,-0.107773,-2.073741,...,-0.804456,0.191491,-0.157713,0.635528,q1,079/080,2079-07-29,20.20,bad,dm2
